# The Travelling Salesperson Problem

This notebook has been adapted from [a Pyevolve example](http://pyevolve.sourceforge.net/0_6rc1/examples.html#example-12-the-travelling-salesman-problem-tsp).

The [travelling salesperson problem (TSP)](http://en.wikipedia.org/wiki/Travelling_salesman_problem) is an NP-hard problem in combinatorial optimization studied in operations research and theoretical computer science. Given a list of cities and their pairwise distances, the task is to find the shortest possible route that visits each city exactly once and returns to the origin city. It is a special case of the travelling purchaser problem.

[<img src="img/travelling_salesman_problem.jpg" align="right" width=360>](http://en.wikipedia.org/wiki/Travelling_salesman_problem)

The code below shows the use of Pyevolve to solve the TSP. Images of the intermediate and final solutions are stored in the 'tspimg' folder.

Your tasks are:
1. Create the 'tspimg' folder for storing the images.
2. Add the necessary statements for storing the results in a database named 'tsp.db' with identifier 'ex1'.
3. For the maximum grade: modify the code to solve the problem with the [ATT 48 dataset](att48.tsp),  a set of 48 cities (US state capitals) from [TSPLIB](http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsplib.html). Store the results in a database named 'tsp_att48.db' with identifier 'ex1'. For your information, [the optimal cost is 10628](http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html).

In [297]:
from pyevolve import G1DList
from pyevolve import GSimpleGA
from pyevolve import Crossovers
from pyevolve import Consts
from pyevolve import DBAdapters

import random
from math import sqrt
from PIL import Image, ImageDraw, ImageFont

In [298]:
cm     = []
coords = []
CITIES = 48
WIDTH   = 600
HEIGHT  = 400
LAST_SCORE = -1

In [299]:
def cartesian_matrix(coords):
    """ A distance matrix """
    matrix={}
    for i,(x1,y1) in enumerate(coords):
        for j,(x2,y2) in enumerate(coords):
            dx, dy = x1-x2, y1-y2
            dist=sqrt(dx*dx + dy*dy)
            matrix[i,j] = dist
    return matrix

In [300]:
def tour_length(matrix, tour):
    """ Returns the total length of the tour """
    total = 0
    t = tour.getInternalList()
    for i in range(CITIES):
        j      = (i+1)%CITIES
        total += matrix[t[i], t[j]]
    return total

In [301]:
def write_tour_to_img(coords, tour, img_file):
    """ The function to plot the graph """
    padding=20
    coords=[(x+padding,y+padding) for (x,y) in coords]
    maxx,maxy=0,0
    for x,y in coords:
        maxx, maxy = max(x,maxx), max(y,maxy)
    maxx+=padding
    maxy+=padding
    img=Image.new("RGB",(int(maxx),int(maxy)),color=(255,255,255))
    font=ImageFont.load_default()
    d=ImageDraw.Draw(img);
    num_cities=len(tour)
    for i in range(num_cities):
        j=(i+1)%num_cities
        city_i=tour[i]
        city_j=tour[j]
        x1,y1=coords[city_i]
        x2,y2=coords[city_j]
        d.line((int(x1),int(y1),int(x2),int(y2)),fill=(0,0,0))
        d.text((int(x1)+7,int(y1)-5),str(i),font=font,fill=(32,32,32))

    for x,y in coords:
        x,y=int(x),int(y)
        d.ellipse((x-5,y-5,x+5,y+5),outline=(0,0,0),fill=(196,196,196))
    del d
    img.save(img_file, "PNG")
    print ("The plot was saved into the %s file." % (img_file,))

In [302]:
def G1DListTSPInitializator(genome, **args):
    """ The initializator for the TSP """
    lst = [i for i in range(genome.getListSize())]
    random.shuffle(lst)
    genome.setInternalList(lst)

In [303]:
def evolve_callback(ga_engine):
    global LAST_SCORE
    if ga_engine.getCurrentGeneration() % 100 == 0:
        best = ga_engine.bestIndividual()
        if LAST_SCORE != best.getRawScore():
            write_tour_to_img( coords, best, "tspimg/tsp_result_%05d.png" % ga_engine.getCurrentGeneration())
            LAST_SCORE = best.getRawScore()
    return False

In [304]:
coords = [(random.randint(0, WIDTH), random.randint(0, HEIGHT))
             for i in range(CITIES)]
cm     = cartesian_matrix(coords)

In [305]:
genome = G1DList.G1DList(len(coords))
genome.evaluator.set(lambda chromosome: tour_length(cm, chromosome))
genome.crossover.set(Crossovers.G1DListCrossoverEdge)
genome.initializator.set(G1DListTSPInitializator)

In [306]:
sqlite_adapter = DBAdapters.DBSQLite(dbname='tsp_att48.db', identify="ex1", resetDB=True)

In [307]:
ga = GSimpleGA.GSimpleGA(genome)
ga.setDBAdapter(sqlite_adapter)
ga.setGenerations(2500)
ga.setMinimax(Consts.minimaxType["minimize"])
ga.setCrossoverRate(0.9)
ga.setMutationRate(0.1)
ga.setPopulationSize(150)
ga.stepCallback.set(evolve_callback)

In [308]:
ga.evolve(freq_stats=200)
best = ga.bestIndividual()
write_tour_to_img(coords, best, "tspimg/tsp_result.png")

The plot was saved into the tspimg/tsp_result_00000.png file.
Gen. 0 (0.00%): Max/Min/Avg Fitness(Raw)             [16662.03(16381.37)/11803.16(12013.56)/13885.03(13885.03)]
The plot was saved into the tspimg/tsp_result_00100.png file.
The plot was saved into the tspimg/tsp_result_00200.png file.
Gen. 200 (8.00%): Max/Min/Avg Fitness(Raw)             [12004.17(12138.56)/7635.43(7476.37)/10003.48(10003.48)]
The plot was saved into the tspimg/tsp_result_00300.png file.
The plot was saved into the tspimg/tsp_result_00400.png file.
Gen. 400 (16.00%): Max/Min/Avg Fitness(Raw)             [11848.48(11659.75)/6325.19(6664.33)/9873.73(9873.73)]
Gen. 600 (24.00%): Max/Min/Avg Fitness(Raw)             [11849.96(11857.45)/6676.46(6664.33)/9874.97(9874.97)]
Gen. 800 (32.00%): Max/Min/Avg Fitness(Raw)             [11831.62(11829.82)/6661.41(6664.33)/9859.69(9859.69)]
The plot was saved into the tspimg/tsp_result_00900.png file.
The plot was saved into the tspimg/tsp_result_01000.png file.
Gen. 1000

You can check now the results by plotting some graphs of the evolution process in [this notebook](TSP_check.ipynb).

In [309]:
ga.step()
print("Generation: %d" % ga.currentGeneration)
best = ga.bestIndividual()
print('Best individual: %s' % str(best.genomeList))
print('Best score: %.0f' % best.score)
population = ga.getPopulation()

Generation: 2501
Best individual: [29, 1, 12, 34, 39, 17, 18, 37, 20, 42, 21, 7, 5, 19, 44, 15, 22, 32, 30, 33, 13, 41, 38, 31, 3, 14, 2, 4, 9, 16, 35, 47, 45, 26, 40, 10, 23, 43, 25, 36, 27, 6, 8, 46, 11, 28, 24, 0]
Best score: 6190
